In [33]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- Setup driver ---
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chromedriver_path = r"C:\Windows\chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# --- Open main shop page ---
shop_url = "https://afroglamourcosmetics.com/"   # replace with your allowed shop URL
driver.get(shop_url)

wait = WebDriverWait(driver, 50)

# --- Collect first 10 product links ---
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.woocommerce-LoopProduct-link")))
product_elements = driver.find_elements(By.CSS_SELECTOR, "a.woocommerce-LoopProduct-link")[:10]
product_links = [el.get_attribute("href") for el in product_elements]

results = []

for link in product_links:
    driver.get(link)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1.product_title")))

    # --- Product name ---
    try:
        name = driver.find_element(By.CSS_SELECTOR, "h1.product_title").text
    except:
        name = ""

    # --- Price ---
    try:
        price = driver.find_element(By.CSS_SELECTOR, "p.price span.woocommerce-Price-amount").text
    except:
        price = ""

    # --- Short description ---
    try:
        desc = driver.find_element(By.CSS_SELECTOR, "div.woocommerce-product-details__short-description").text
    except:
        desc = ""

    # --- Breadcrumb (brand/category) ---
    try:
        crumbs = driver.find_elements(By.CSS_SELECTOR, "nav.woocommerce-breadcrumb a")
        brand = crumbs[-1].text if crumbs else ""
    except:
        brand = ""

    # --- Images (only from product gallery) ---
    img_urls = []
    try:
        imgs = driver.find_elements(By.CSS_SELECTOR, "figure.woocommerce-product-gallery__image img")
        img_urls = [img.get_attribute("src") for img in imgs if img.get_attribute("src")]
    except:
        pass

    results.append({
        "Product URL": link,
        "Brand": brand,
        "Product name": name,
        "Ingredients": "",         # placeholder unless you know selector
        "Product Image": img_urls,
        "Price": price,
        "Description": desc,
        "Product Line Name": "",
        "Barcode": "",
        "Size/Volume": "",
        "Skin Concern": ""
    })

Build DataFrame

In [34]:
product_df = pd.DataFrame(results)
globals()["product_df"] = product_df

Save to CSV

In [37]:

save_path = r"C:\Users\USER\Documents\Script\products.csv"
product_df.to_csv(save_path, index=False, encoding="utf-8")

print(f"Saved {len(product_df)} products to {save_path}")

print(product_df.head())

Saved 10 products to C:\Users\USER\Documents\Script\products.csv
                                         Product URL  \
0  https://afroglamourcosmetics.com/product/caro-...   
1  https://afroglamourcosmetics.com/product/caro-...   
2  https://afroglamourcosmetics.com/product/makar...   
3  https://afroglamourcosmetics.com/product/makar...   
4  https://afroglamourcosmetics.com/product/makar...   

                                Brand  \
0                Lightening Body Oils   
1                Lightening Body Oils   
2                        BLUE CRYSTAL   
3                        BLUE CRYSTAL   
4  NATURALLE CAROTONIC EXTREME SPF 15   

                                  Product name Ingredients Product Image  \
0                    Caro White Lightening Set                        []   
1                    Caro White Lightening Set                        []   
2            Makari Blue Crystal 4-Pc Gift Set                        []   
3            Makari Blue Crystal 4-Pc Gift Set 